In [1]:
from IPython.display import Latex
import latexStrings as ls
import numpy as np
import scipy.linalg as linear

def simpleQR(A, tol, maxIter=1e3, symmetric=False):
    i = 0
    if not symmetric:
        while i < maxIter and linear.norm(A.diagonal(-1))>tol:
            [Q,R] = linear.qr(A)
            A = R.dot(Q)
            i += 1
        return A,i
    else:
        V=np.identity(len(A))
        while i < maxIter and linear.norm(A.diagonal(-1))>tol:
            [Q,R] = linear.qr(A)
            A = R.dot(Q)
            V = V.dot(Q)
            i += 1
        return A,V,i

def shiftQRStep(A, tol, symmetric=False):
    i = 0
    if not symmetric:
        while abs(A[-1,-2])>tol:
            shift = A[-1,-1]
            [Q,R] = linear.qr(A-shift*np.identity(len(A)))
            A = R.dot(Q)+shift*np.identity(len(A))
            i += 1
        return A,i
    else:
        #Acumular eigenvectores
        while abs(A[-1,-2])>tol:
            delta = 0.5*(A[-2,-2]-A[-1,-1])
            shift = A[-1,-1] - np.sign(delta)*((A[-2,-1]**2)/(abs(delta)+np.sqrt(delta**2+A[-2,-1]**2)))
            [Q,R] = linear.qr(A-shift*np.identity(len(A)))
            A = R.dot(Q)+shift*np.identity(len(A))
            i += 1
        return A,i

def shiftQR(A, tol, maxIter=1e3, symmetric=False):
    i = 0
    A = linear.hessenberg(A)
    eigenvalues = []
    if not symmetric:
        while i<maxIter and len(A)>1:
            [A,j] = shiftQRStep(A,tol)
            eigenvalues.append(A[-1,-1])
            A = A[:-1,:-1]
            i += j
        eigenvalues.append(A[0,0])
        return np.array(eigenvalues),i
    else:
        while i<maxIter and len(A)>1:
            [A,j] = shiftQRStep(A,tol, True)
            eigenvalues.append(A[-1,-1])
            A = A[:-1,:-1]
            i += j
        eigenvalues.append(A[0,0])
        return np.array(eigenvalues),i

In [8]:
A=np.array([[0,1,2],[1,0,1],[2,1,0]])

a=ls.latexMatrix(A,'A')
Latex(a)

<IPython.core.display.Latex object>

In [3]:
[lam,V]=linear.eig(A)

l=ls.latexVector(lam,'\lambda', form='%0.04f')
v=ls.latexMatrix(V,'V', form='%0.4f')
Latex(l+'\n'+v)

<IPython.core.display.Latex object>

In [4]:
[T,iterations] = simpleQR(A,1e-10)

t=ls.latexMatrix(T,'T', form='%0.4f')
Latex('Iteraciones totales: '+str(iterations)+'\n'+t)

<IPython.core.display.Latex object>

In [5]:
[T,iterations] = shiftQRStep(A,1e-10)

t = ls.latexMatrix(T,'T', form='%0.4f')
Latex('Iteraciones totales: '+str(iterations)+'\n'+t)

<IPython.core.display.Latex object>

In [6]:
[lam,iteraciones]=shiftQR(A,1e-10)

l=ls.latexVector(lam,'\lambda', form='%0.04f')
Latex('Iteraciones totales: ' + str(iterations) +'\n' + l)

<IPython.core.display.Latex object>

In [9]:
B=np.array([[0,1,2],[1,0,1],[2,1,0]])

b=ls.latexMatrix(B,'B')
Latex(b)

<IPython.core.display.Latex object>

In [10]:
[T,V,iterations]=simpleQR(B,1e-10,symmetric=True)

v=ls.latexMatrix(V,'V', form='%0.4f')
t=ls.latexMatrix(T,'T', form='%0.4f')
Latex('Iteraciones totales: '+ str(iterations) + '\n' + t + '\n' + v)

<IPython.core.display.Latex object>

In [11]:
[T,iterations]=shiftQRStep(B,1e-10, symmetric=True)

t = ls.latexMatrix(T,'T', form='%0.4f')
Latex('Iteraciones totales: '+str(iterations)+'\n'+t)

<IPython.core.display.Latex object>

In [12]:
[lam,iterations] = shiftQR(B,1e-10, symmetric=True)

l=ls.latexVector(lam,'\lambda', form='%0.04f')
Latex('Iteraciones totales: ' + str(iterations) +'\n' + l)

<IPython.core.display.Latex object>